In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import sys, os
pbin_dir = os.path.dirname(sys.executable)
%env PBIN_DIR={pbin_dir}
os.environ['PATH'] = f'{os.environ["PBIN_DIR"]}:{os.environ["PATH"]}'
print(os.environ['PATH'])

In [ ]:
#オリジナル
#%%capture output
#output.show()で見れる or ファイルが出力されているか実際に確認する。

import pydash
import pickle
#vocab, word_embeddings = pickle.load(file=open(f"dataset/reddit/vocab_and_embeddings.pkl", 'rb'))
#cemb
#vocab, word_embeddings = pickle.load(file=open(f"vocab_and_embeddings_redpol.pkl", 'rb'))
vocab, word_embeddings = pickle.load(file=open(f"vocab_and_embeddings_nfl_minill3.pkl", 'rb'))
#vocab, word_embeddings = pickle.load(file=open(f"vocab_and_embeddings_politics_minill3.pkl", 'rb'))
import torchtext
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english")
#cemb
unkkey=list(vocab.keys())[-1]
#unkkey='unk'
print(f'unkkey: {unkkey}')
unkv = vocab[unkkey]
print(f'unkv: {unkv}')
import json
import random
def conv_uttr(pe,seqlen,vocab):
    ts = [tokenizer(x) for x in pe]
    if any([len(x) > seqlen for x in ts]):
        return None
    #以下3行はmin3,min3top2000の計測の際は入れていた
    #if any(['(' in x for x in ts]):
    #    continue
    #ts = [[y.replace('\\n', '').strip() for y in x] for x in ts]
    #ts = [[y for y in x if y != ''] for x in ts]
    ts = [['<PAD>'] * (seqlen - len(x)) + x for x in ts]
    ts = [[vocab.get(y, vocab[unkkey]) for y in x] for x in ts]
    return ts
def create_test_dict(tsv,atsv):
    tsvd = []
    atsvd = []
    ret = {}
    with open(tsv) as f:
        tsvd = [x.split('\t') for x in f.read().strip().splitlines()]
    with open(atsv) as f:
        atsvd = [x.split('\t') for x in f.read().strip().splitlines()]
    for d, a in zip(tsvd,atsvd):
        ca = a[-2]
        if ca not in ret:
            ret[ca] = {'p':[],'n':[]}
        ret[ca]['p' if d[0] == '1' else 'n'].append([d[1:],a])
    for retk in ret:
        retp = ret[retk]['p']
        retn = ret[retk]['n']
        assert(len(retp) == 1)
        assert(len(retn) == 9)
        for retne in retn:
            if retp[0][0][-1] == retne[0][-1]:
                print(retp[0])
                print(retne)
                #politicsの場合おなじresponseのものがtestデータに1件ある
                #assert(False)
    return ret

def create_data(d,u,pklname,tsv=None,atsv=None):
    #以下はmin3,min3top2000のときは50としていた
    seqlen = 50
    ud = []
    rdt = []
    yd = []
    debugd = []
    unkc = 0
    tc = 0
    er1 = 0
    er2 = 0
    er3 = 0
    keys = d.keys()
    if u is not None:
        keys = u
    print(f'keyc: {len(keys)}')
    print(keys[0])#author
    
    for k in keys:
        hs = d[k]#authorの履歴
        #min3,min3top2000の計測の際は以下を入れてた
        ude = []
        #min3,min3top2000の計測の際は通常のresponse authorをcontext authorとみなしてた
        ohs = hs#なんのために変数をわざわざ変える?
        #print(len(hs))
        #print(len(ohs))
        oh = ohs[-1]#authorの最新履歴
#         print(oh)
#         print()
        ps = []
        roh = oh['response_author']#authorの最新履歴のレスポンス側のauthor
          
        if roh in d:#jsonのauthorキーにresponse_authorがいたら、
            hs = d[roh]#response_authorの履歴リストを取得
            #hs = hs[:3]#debug
            hs.sort(key=lambda x: int(x['context_created_utc']))#contextの時刻でソート
            #uniqhist
            hs = pydash.uniq_by(hs,lambda x: x['Context'])
            for hi, h in enumerate(hs):#取得したresponse_authorの履歴から取得したデータをすべてpsにappendする
                #print(hi)#history index
                #print(h)#履歴
                
                if h['context_created_utc'] >= oh['response_created_utc']:#response_authorの履歴のcontext時間が、authorの最新履歴のcontext時間以上(新しい)場合はcontinue?
                    continue
                if 'context/0' not in h:#response_authorの履歴にcontext/0がない場合continue
                    continue
                ps.append([h[x] for x in ['context/0','Context']])#response_authorの履歴のcontext/0とContextをpsにappend
        ps.append([oh[x] for x in ['Context', 'Response']])#authorの履歴のContextとResponseをpsにappend
#         print(len(ps))
#         print()
        
#         print(ps)
#         print(dsa)
        er2o=False
        
        for pi, pe in enumerate(ps):#上記処理で取得したresponse_author履歴+authorの最新履歴のリスト
            #print(pe)
            ts = conv_uttr(pe,seqlen,vocab)#id化、(先頭indexから0埋めされている)
            ###　例
            #[
            #[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 459, 40, 401, 443, 26, 35, 283],
            #[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60, 208, 12, 3, 18, 11125, 19, 10494, 2, 221, 10, 438, 58, 8, 822]
            #]
            ###
            #print(ts)
            
            if ts is None:#conv_utterの結果がnoneのときのスキップフラグオン？
                if pi == len(ps) - 1:
                    er2+=1
                    er2o=True
                continue
                
            for t in ts:
                unkc += len([x for x in t if x == unkv])#unknownIDがあれば加算？
                tc += len(t)#seqの長さを足す(合計100になるはず)
                ude.append(t)
            
        if er2o:
            continue
            
        if len(ude) == 0:
            er3 +=1
            continue
        
#         print(ude)
#         print()
        resp = ude[-1]#response側？
#         print(resp)
        ude = ude[-30:-1]#最新から29個分？
#         print(ude)
        ude = [*([[0] * seqlen] * (29 - len(ude))), *ude]

        
        for i in range(10):
            debugd.append([k,roh])
            if i == 0:
                yd.append(1)
                #デバッグ用にrawレスポンスとresponse authorを仕込む
                rdt.append([resp,k,ps[-1][1],roh])
            else:
                yd.append(0)
            ud.append(ude)#加工した29データを10個分用意している
        
 
        #1authorの履歴からのデータ処理はここまで
            
        ### udについて
        #[ud,rd,yd]がpickleになってる
        #ydは0 or 1のarray
        #rdは当てに行く返答。n個に1個正解（yd=1）がある
        
        #udはrdを当てるためのデータ。長さ29のarrayで中身は発話で、最後の要素以外はだれか（だれかは毎回異なって良い）の発話とそれに対するrdの返答者による返答となる。
        #最後の要素はだれかの発話となり、rdの返答はこの最後のだれかの発話への返答となる。長さが余る場合は後ろから埋めて、あまりを空発話にする
        #返答・発話は、長さが固定（seqlen）のarrayで中身はtoken id(integer)。余る場合は先頭からに対応するidで埋める。空発話であれば、全てに対応するidとなる
        #
            
    test_dict=None
    if tsv is not None:
        test_dict=create_test_dict(tsv,atsv)
    rd = []
    for rdi, rde in enumerate(rdt):
        rd.append(rde[0])
        if test_dict is None:
            for i in range(9):
                sr = list(range(len(rdt)))
                sr.remove(rdi)
                si = random.choice(sr)
                rd.append(rdt[si][0])
        else:
            test_dict_ent = test_dict[rde[1]]
            tdep = test_dict_ent['p'][0]
            tdepd,tdepa=tdep
            assert(tdepa[-2]==rde[1])
            assert(tdepa[-1]==rde[3])
            try:
                assert(all([x==y for x,y in zip(conv_uttr([tdepd[-1]],seqlen,vocab)[0],rde[0])]))
            except:
                print(len(rd))
                print(tdepd)
                print(tdepa)
                print(conv_uttr([tdepd[-1]],seqlen,vocab)[0])
                print(rde)
                assert(False)
            assert(yd[rdi*10] == 1)
            for tdei, tde in enumerate(test_dict_ent['n']):
                assert(len(rd)%10==tdei+1)
                assert(yd[rdi*10+tdei+1] == 0)
                d,a=tde
                assert(a[-2]==rde[1])
                ts = conv_uttr([d[-1]],seqlen,vocab)
                assert(ts is not None)
                rd.append(ts[0])
            #frd=rd[-10]
            #for rrdi, rrd in enumerate(rd[-9:]):
                #unkによって1wordのresponseが同じにみなされることはある
                #if all([x == y for x,y in zip(rrd,frd)]):
                #    print(rde[1])
                #    print(rde[2])
                #    print(test_dict_ent['n'][rrdi][0][-1])
                #    print(rrdi)
                #    print(frd)
                #    print(rrd)
                #    assert(False)
    print(f'unkc: {unkc}')
    print(f'tc: {tc}')
    print(f'unkc/tc: {unkc/tc}')
    print(f'{er1},{er2},{er3}')
    assert(er1==0)
    #politicsの場合trainデータにあるっぽい
    #assert(er2==0)
    assert(er3==0)
    assert(len(ud)==len(rd))
    assert(len(ud)==len(yd))
    with open(pklname,'wb') as f:
        pickle.dump([ud,rd,yd],f)
    with open(f'{pklname}_debug.pkl','wb') as f:
        pickle.dump(debugd,f)
    a,b,c = pickle.load(file=open(pklname, 'rb'))

d = None
#with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_politics_minill3_test.json') as f:
with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_nfl_minill3_test.json') as f:
# with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_politics_minill3_test.json') as f:
    d = json.loads(f.read())
    
u = None
#with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_201809101112_20190102_politics_minill3_test_top10000.txt') as f:
with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_201809101112_20190102_nfl_minill3_test_top3000.txt') as f:
# with open('/share_7/projects/hais/data/reddit_nfl/author_list_min30_201809101112_20190102_politics_minill3_test.txt') as f:
# with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_201809101112_20190102_politics_minill3_test.txt') as f:
    u = f.read().strip().split('\n')
#create_data(d, u,
#            'tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl',
#            '/share_7/projects/hais/data/reddit_nfl_bertfp/minill3politics_data/201809101112_20190102_politics_minill3_test.tsv',
#            '/share_7/projects/hais/data/reddit_nfl_bertfp/minill3politics_data/201809101112_20190102_politics_minill3_test_authors.tsv')

#create_data(d, u,
#           'tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl',
#           '/share_7/projects/hais/data/reddit_nfl_bertfp/minill3nfl10_data/201809101112_20190102_nfl_minill3_test.tsv',
#           '/share_7/projects/hais/data/reddit_nfl_bertfp/minill3nfl10_data/201809101112_20190102_nfl_minill3_test_authors.tsv')

# create_data(d, u,
#            'tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl',
#            '/share_7/projects/hais/data/reddit_nfl_bertfp/minill3politics_data/201809101112_20190102_politics_minill3_test.tsv',
#            '/share_7/projects/hais/data/reddit_nfl_bertfp/minill3politics_data/201809101112_20190102_politics_minill3_test_authors.tsv')


#create_data(d, u, 'tm_test_min20_clean_bugfix_try.pkl')
with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_nfl_minill3_train.json') as f:
# with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_nfl_minill3_train.json') as f:
#with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_politics_minill3_train.json') as f:
    d = json.loads(f.read())
#u = None #もともと

#with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_201809101112_20190102_politics_minill3_train_top10000.txt') as f:
with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_11503_201809101112_20190102_nfl_minill3_train_top500.txt') as f:
#with open('/share_7/projects/hais/data/reddit_nfl/train_reddit_politics_top3000_ctxloop50_resloop50.txt') as f:
# with open('/share_7/projects/hais/data/reddit_nfl/train_reddit_nlf_top3000_ctxloop50_resloop50.txt') as f:

# with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_1983_201809101112_20190102_nfl_minill3_train.txt') as f:
# with open('/share_7/projects/hais/data/reddit_nfl/author_list_min30_9619_201809101112_20190102_politics_minill3_train.txt') as f:
#with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_14501_201809101112_20190102_politics_minill3_train.txt') as f:
    u = f.read().strip().split('\n')
    
create_data(d, u, 'tm_train_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl')



# create_data(d, u, 
#             'tm_train_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl',
#            '/share_7/projects/hais/data/reddit_nfl_bertfp/minill3nfl10_data/201809101112_20190102_nfl_minill3_train.tsv',
#            '/share_7/projects/hais/data/reddit_nfl_bertfp/minill3nfl10_data/201809101112_20190102_nfl_minill3_train_authors.tsv')


In [ ]:

#%%capture output
#ループカウントとスライド実装版
#output.show()で見れる or ファイルが出力されているか実際に確認する。

import pydash
import pickle
#vocab, word_embeddings = pickle.load(file=open(f"dataset/reddit/vocab_and_embeddings.pkl", 'rb'))
#cemb
#vocab, word_embeddings = pickle.load(file=open(f"vocab_and_embeddings_redpol.pkl", 'rb'))
#vocab, word_embeddings = pickle.load(file=open(f"vocab_and_embeddings_nfl_minill3.pkl", 'rb'))
vocab, word_embeddings = pickle.load(file=open(f"vocab_and_embeddings_politics_minill3.pkl", 'rb'))
import torchtext
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english")
#cemb
unkkey=list(vocab.keys())[-1]
#unkkey='unk'
print(f'unkkey: {unkkey}')
unkv = vocab[unkkey]
print(f'unkv: {unkv}')
import json
import random
from tqdm.auto import tqdm


def conv_uttr(pe,seqlen,vocab):
    ts = [tokenizer(x) for x in pe]
    if any([len(x) > seqlen for x in ts]):
        return None
    #以下3行はmin3,min3top2000の計測の際は入れていた
    #if any(['(' in x for x in ts]):
    #    continue
    #ts = [[y.replace('\\n', '').strip() for y in x] for x in ts]
    #ts = [[y for y in x if y != ''] for x in ts]
    ts = [['<PAD>'] * (seqlen - len(x)) + x for x in ts]
    ts = [[vocab.get(y, vocab[unkkey]) for y in x] for x in ts]
    return ts
def create_test_dict(tsv,atsv):
    tsvd = []
    atsvd = []
    ret = {}
    with open(tsv) as f:
        tsvd = [x.split('\t') for x in f.read().strip().splitlines()]
    with open(atsv) as f:
        atsvd = [x.split('\t') for x in f.read().strip().splitlines()]
    for d, a in zip(tsvd,atsvd):
        ca = a[-2]
        if ca not in ret:
            ret[ca] = {'p':[],'n':[]}
        ret[ca]['p' if d[0] == '1' else 'n'].append([d[1:],a])
    for retk in ret:
        retp = ret[retk]['p']
        retn = ret[retk]['n']
        assert(len(retp) == 1)
        assert(len(retn) == 9)
        for retne in retn:
            if retp[0][0][-1] == retne[0][-1]:
                print(retp[0])
                print(retne)
                #politicsの場合おなじresponseのものがtestデータに1件ある
                #assert(False)
    return ret

debug_author=[]
debug_response_author=[]
def create_data(d,u,pklname,tsv=None,atsv=None,loop_count=1,max_slide_num=1):
    global debug_author
    global debug_response_author
    #以下はmin3,min3top2000のときは50としていた
    seqlen = 50
    ud = []
    rdt = []
    yd = []
    debugd = []
    unkc = 0
    tc = 0
    er1 = 0
    er2 = 0
    er3 = 0
    keys = d.keys()
    if u is not None:
        keys = u
    print(f'keyc: {len(keys)}')#author
    
    for k in tqdm(keys,desc="create_data"):
        hs = d[k]#authorの履歴
        for start in range(max_slide_num):
            num_histories = len(hs)
            histories = hs[
            max([num_histories - loop_count - start, 0]):max([num_histories - start, 0])
                                           ]#最新の履歴から見てloopcount分スライドしながら取り出す
            #print([x for x in range(max([num_histories - loop_count - start, 0]),max([num_histories - start, 0]))])
            #print(len([x for x in range(max([num_histories - loop_count - start, 0]),max([num_histories - start, 0]))]))
            num_histories = len(histories)
            if num_histories != loop_count:
                # 取り出した履歴データがloopcount分に満たない場合はスキップ
                #print("skip")
                continue
            #histories = hs[-loop_count:]#loop_count分の履歴を対象にデータ処理
            for history in histories:
                #min3,min3top2000の計測の際は以下を入れてた
                ude = []
                #min3,min3top2000の計測の際は通常のresponse authorをcontext authorとみなしてた
                #ohs = hs#なんのために変数をわざわざ変える?
                #oh = ohs[-1]#authorの最新履歴
                oh = history
                ps = []
                roh = oh['response_author']#historyのレスポンス側のauthor


                if roh in d:#jsonのauthorキーにresponse_authorがいたら、
                    hs = d[roh]#response_authorの履歴リストを取得
                    #hs = hs[:3]#debug
                    hs.sort(key=lambda x: int(x['context_created_utc']))#contextの時刻でソート
                    #uniqhist
                    hs = pydash.uniq_by(hs,lambda x: x['Context'])
                    for hi, h in enumerate(hs):#取得したresponse_authorの履歴から取得したデータをすべてpsにappendする
                        #print(hi)#history index
                        #print(h)#履歴

                        if h['context_created_utc'] >= oh['response_created_utc']:#response_authorの履歴のcontext時間が、authorの最新履歴のcontext時間以上(新しい)場合はcontinue?
                            continue
                        if 'context/0' not in h:#response_authorの履歴にcontext/0がない場合continue
                            continue
                        ps.append([h[x] for x in ['context/0','Context']])#response_authorの履歴のcontext/0とContextをpsにappend 誰かの発話とrdの返答(Context)
                ps.append([oh[x] for x in ['Context', 'Response']])#authorの履歴のContextとResponseをpsにappend

                er2o=False

                for pi, pe in enumerate(ps):#上記処理で取得したresponse_author履歴+authorの最新履歴のリスト
                    ts = conv_uttr(pe,seqlen,vocab)#id化、(先頭indexから0埋めされている?)
                    ###　例
                    #[
                    #[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 459, 40, 401, 443, 26, 35, 283],
                    #[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60, 208, 12, 3, 18, 11125, 19, 10494, 2, 221, 10, 438, 58, 8, 822]
                    #]
                    ###
                    #print(ts)

                    if ts is None:#conv_utterの結果がnoneのときのスキップフラグオン？
                        if pi == len(ps) - 1:
                            er2+=1
                            er2o=True
                        continue

                    for t in ts:
                        unkc += len([x for x in t if x == unkv])#unknownIDがあれば加算？
                        tc += len(t)#seqの長さを足す(合計100になるはず)
                        ude.append(t)

                if er2o:
                    continue

                if len(ude) == 0:
                    er3 +=1
                    continue

                resp = ude[-1]#response側？
                ude = ude[-30:-1]#反転させているが、なぜ-30から-1なのか不明
                ude = [*([[0] * seqlen] * (29 - len(ude))), *ude]


                for i in range(10):
                    debugd.append([k,roh])
                    if i == 0:
                        yd.append(1)#ラベル
                        #デバッグ用にrawレスポンスとresponse authorを仕込む
                        rdt.append([resp,k,ps[-1][1],roh])#正解データ
                    else:
                        yd.append(0)
                    ud.append(ude)#加工した29データを10個分用意している
                    
                debug_author.append(k)
                debug_response_author.append(roh)
                
    print(len(ud))
    print(len(yd))
    print(len(rdt))
    
    test_dict=None
    if tsv is not None:
        test_dict=create_test_dict(tsv,atsv)
    rd = []
    for rdi, rde in tqdm(enumerate(rdt),desc="rdt"):
        rd.append(rde[0])
        if test_dict is None:
            #train
            #不正解データ9個
            for i in range(9):
                sr = list(range(len(rdt)))
                sr.remove(rdi)
                si = random.choice(sr)
                rd.append(rdt[si][0])
        else:
            #test valid
            test_dict_ent = test_dict[rde[1]]
            tdep = test_dict_ent['p'][0]
            tdepd,tdepa=tdep
            assert(tdepa[-2]==rde[1])
            assert(tdepa[-1]==rde[3])
            try:
                assert(all([x==y for x,y in zip(conv_uttr([tdepd[-1]],seqlen,vocab)[0],rde[0])]))
            except:
                print(len(rd))
                print(tdepd)
                print(tdepa)
                print(conv_uttr([tdepd[-1]],seqlen,vocab)[0])
                print(rde)
                assert(False)
            assert(yd[rdi*10] == 1)
            for tdei, tde in enumerate(test_dict_ent['n']):
                assert(len(rd)%10==tdei+1)
                assert(yd[rdi*10+tdei+1] == 0)
                d,a=tde
                assert(a[-2]==rde[1])
                ts = conv_uttr([d[-1]],seqlen,vocab)
                assert(ts is not None)
                rd.append(ts[0])

    print(f'unkc: {unkc}')
    print(f'tc: {tc}')
    print(f'unkc/tc: {unkc/tc}')
    print(f"ud:{len(ud)}")
    print(f"rd:{len(rd)}")
    print(f"yd:{len(yd)}")
    print(f'{er1},{er2},{er3}')
    print(f"author:{len(list(set(debug_author)))}")
    print(f"response_author:{len(list(set(debug_response_author)))}")


    assert(er1==0)
    #politicsの場合trainデータにあるっぽい
    #assert(er2==0)
    assert(er3==0)
    assert(len(ud)==len(rd))
    assert(len(ud)==len(yd))
    with open(pklname,'wb') as f:
        pickle.dump([ud,rd,yd],f)
    with open(f'{pklname}_debug.pkl','wb') as f:
        pickle.dump(debugd,f)
    print("pkl saved")
    a,b,c = pickle.load(file=open(pklname, 'rb'))
    print("done")

#test


#with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_nfl_minill3_test.json') as f:
with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_politics_minill3_test.json') as f:
    d = json.loads(f.read())
    
# with open("/share_7/projects/hais/data/reddit_nfl/author_list_min20_13765_201809101112_20190102_nfl_minill3_test_top500.txt") as f:
#with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_201809101112_20190102_nfl_minill3_test.txt') as f:
with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_201809101112_20190102_politics_minill3_test.txt') as f:
     u = f.read().strip().split('\n')

create_data(d,u,
            "tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl",
            "/share_7/projects/hais/data/reddit_nfl_bertfp/minill3politics_data/201809101112_20190102_politics_minill3_test.tsv",
            "/share_7/projects/hais/data/reddit_nfl_bertfp/minill3politics_data/201809101112_20190102_politics_minill3_test_authors.tsv",
            1,1
           )
# create_data(d,u,
#             "tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl",
#             "/share_7/projects/hais/data/reddit_nfl_bertfp/minill3nfl10_data/201809101112_20190102_nfl_minill3_test.tsv",
#             "/share_7/projects/hais/data/reddit_nfl_bertfp/minill3nfl10_data/201809101112_20190102_nfl_minill3_test_authors.tsv",
#             1,1
#            )


#train
#with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_nfl_minill3_train.json') as f:
with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_politics_minill3_train.json') as f:
    d = json.loads(f.read())

# with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_11503_201809101112_20190102_nfl_minill3_train_top500.txt') as f:
#with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_1983_201809101112_20190102_nfl_minill3_train.txt') as f:
with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_14501_201809101112_20190102_politics_minill3_train.txt') as f:
    u = f.read().strip().split('\n')
    

create_data(d, u, 
            'tm_train_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl',
            None,
            None,
            3,20)






In [ ]:

#%%capture output
#slide分の履歴を対象に19件のペルソナ版

import pydash
import pickle
#vocab, word_embeddings = pickle.load(file=open(f"dataset/reddit/vocab_and_embeddings.pkl", 'rb'))
#cemb
#vocab, word_embeddings = pickle.load(file=open(f"vocab_and_embeddings_redpol.pkl", 'rb'))
vocab, word_embeddings = pickle.load(file=open(f"vocab_and_embeddings_nfl_minill3.pkl", 'rb'))
#vocab, word_embeddings = pickle.load(file=open(f"vocab_and_embeddings_politics_minill3.pkl", 'rb'))
import torchtext
from torchtext.data import get_tokenizer
tokenizer = get_tokenizer("basic_english")
#cemb
unkkey=list(vocab.keys())[-1]
#unkkey='unk'
print(f'unkkey: {unkkey}')
unkv = vocab[unkkey]
print(f'unkv: {unkv}')
import json
import random
from tqdm.auto import tqdm


def conv_uttr(pe,seqlen,vocab):
    ts = [tokenizer(x) for x in pe]
    if any([len(x) > seqlen for x in ts]):
        return None
    #以下3行はmin3,min3top2000の計測の際は入れていた
    #if any(['(' in x for x in ts]):
    #    continue
    #ts = [[y.replace('\\n', '').strip() for y in x] for x in ts]
    #ts = [[y for y in x if y != ''] for x in ts]
    ts = [['<PAD>'] * (seqlen - len(x)) + x for x in ts]
    ts = [[vocab.get(y, vocab[unkkey]) for y in x] for x in ts]
    return ts
def create_test_dict(tsv,atsv):
    tsvd = []
    atsvd = []
    ret = {}
    with open(tsv) as f:
        tsvd = [x.split('\t') for x in f.read().strip().splitlines()]
    with open(atsv) as f:
        atsvd = [x.split('\t') for x in f.read().strip().splitlines()]
    for d, a in zip(tsvd,atsvd):
        ca = a[-2]
        if ca not in ret:
            ret[ca] = {'p':[],'n':[]}
        ret[ca]['p' if d[0] == '1' else 'n'].append([d[1:],a])
    for retk in ret:
        retp = ret[retk]['p']
        retn = ret[retk]['n']
        assert(len(retp) == 1)
        assert(len(retn) == 9)
        for retne in retn:
            if retp[0][0][-1] == retne[0][-1]:
                print(retp[0])
                print(retne)
                #politicsの場合おなじresponseのものがtestデータに1件ある
                #assert(False)
    return ret


def create_data(d,u,pklname,tsv=None,atsv=None,loop_count=1):
    debug_author=[]
    debug_response_author=[]
    #以下はmin3,min3top2000のときは50としていた
    seqlen = 50
    ud = []
    rdt = []
    yd = []
    debugd = []
    unkc = 0
    tc = 0
    er1 = 0
    er2 = 0
    er3 = 0
    keys = d.keys()
    
    if u is not None:
        keys = u
    print(f'keyc: {len(keys)}')#author
    
    for k in tqdm(keys,desc="create_data"):
        hs = d[k]#authorの履歴
        
        histories = hs[-loop_count:]#loop_count分の履歴を対象にデータ処理
        debug_author.append(k)
        debug_response_author.append(histories[-1]["response_author"])
        for history in histories:
            ude = []
            oh = history
            ps = []
            roh = oh['response_author']#historyのレスポンス側のauthor

            if roh in d:#jsonのauthorキーにresponse_authorがいたら、
                hs = d[roh]#response_authorの履歴リストを取得
                #hs = hs[:3]#debug
                hs.sort(key=lambda x: int(x['context_created_utc']))#contextの時刻でソート
                #uniqhist
                hs = pydash.uniq_by(hs,lambda x: x['Context'])
                for hi, h in enumerate(hs):#取得したresponse_authorの履歴から取得したデータをすべてpsにappendする
                    #print(hi)#history index
                    #print(h)#履歴
                    if h['context_created_utc'] >= oh['response_created_utc']:#response_authorの履歴のcontext時間が、authorの最新履歴のcontext時間以上(新しい)場合はcontinue?
                        continue
                    if 'context/0' not in h:#response_authorの履歴にcontext/0がない場合continue
                        continue
                    ps.append([h[x] for x in ['context/0','Context']])#response_authorの履歴のcontext/0とContextをpsにappend 誰かの発話とrdの返答(Context)
            ps.append([oh[x] for x in ['Context', 'Response']])#authorの履歴のContextとResponseをpsにappend

            er2o=False

            for pi, pe in enumerate(ps):#上記処理で取得したresponse_author履歴+authorの最新履歴のリスト
                ts = conv_uttr(pe,seqlen,vocab)#id化、(先頭indexから0埋めされている?)
                ###　例
                #[
                #[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 459, 40, 401, 443, 26, 35, 283],
                #[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60, 208, 12, 3, 18, 11125, 19, 10494, 2, 221, 10, 438, 58, 8, 822]
                #]
                ###
                #print(ts)

                if ts is None:#conv_utterの結果がnoneのときのスキップフラグオン？
                    if pi == len(ps) - 1:
                        er2+=1
                        er2o=True
                    continue

                for t in ts:
                    unkc += len([x for x in t if x == unkv])#unknownIDがあれば加算？
                    tc += len(t)#seqの長さを足す(合計100になるはず)
                    ude.append(t)

            if er2o:
                continue

            if len(ude) == 0:
                er3 +=1
                continue
                
            ud_num = 19 #rdを当てに行くデータの数 偶数だと学習できない
            
            resp = ude[-1]#response側？
            #ude = ude[-30:-1]#-30から-1 = 29件のデータに絞る
            #ude = [*([[0] * seqlen] * (29 - len(ude))), *ude]
            ude = ude[-ud_num+1:-1]
            ude = [*([[0] * seqlen] * (ud_num - len(ude))), *ude]

            for i in range(10):
                debugd.append([k,roh])
                if i == 0:
                    yd.append(1)#ラベル
                    #デバッグ用にrawレスポンスとresponse authorを仕込む
                    rdt.append([resp,k,ps[-1][1],roh])#正解データ
                else:
                    yd.append(0)
                ud.append(ude)#加工した29件データを10個分用意している

            
                
    print(len(ud))
    print(len(yd))
    print(len(rdt))
    
    test_dict=None
    if tsv is not None:
        test_dict=create_test_dict(tsv,atsv)
    rd = []
    for rdi, rde in tqdm(enumerate(rdt),desc="rdt"):
        rd.append(rde[0])
        if test_dict is None:
            #train
            #不正解データ9個?
            for i in range(9):
                sr = list(range(len(rdt)))
                sr.remove(rdi)
                si = random.choice(sr)
                rd.append(rdt[si][0])
        else:
            #test valid
            test_dict_ent = test_dict[rde[1]]
            tdep = test_dict_ent['p'][0]
            tdepd,tdepa=tdep
            assert(tdepa[-2]==rde[1])
            assert(tdepa[-1]==rde[3])
            try:
                assert(all([x==y for x,y in zip(conv_uttr([tdepd[-1]],seqlen,vocab)[0],rde[0])]))
            except:
                print(len(rd))
                print(tdepd)
                print(tdepa)
                print(conv_uttr([tdepd[-1]],seqlen,vocab)[0])
                print(rde)
                assert(False)
            assert(yd[rdi*10] == 1)
            for tdei, tde in enumerate(test_dict_ent['n']):
                assert(len(rd)%10==tdei+1)
                assert(yd[rdi*10+tdei+1] == 0)
                d,a=tde
                assert(a[-2]==rde[1])
                ts = conv_uttr([d[-1]],seqlen,vocab)
                assert(ts is not None)
                rd.append(ts[0])

    print(f'unkc: {unkc}')
    print(f'tc: {tc}')
    print(f'unkc/tc: {unkc/tc}')
    print(f"ud:{len(ud)}")
    print(f"ud_num:{len(ud[0])}")
    print(f"rd:{len(rd)}")
    print(f"yd:{len(yd)}")
    print(f'{er1},{er2},{er3}')
    print(f"author:{len(list(set(debug_author)))}")
    print(f"response_author:{len(list(set(debug_response_author)))}")
    

    assert(er1==0)
    #politicsの場合trainデータにあるっぽい
    #assert(er2==0)
    assert(er3==0)
    assert(len(ud)==len(rd))
    assert(len(ud)==len(yd))
    with open(pklname,'wb') as f:
        pickle.dump([ud,rd,yd],f)
    with open(f'{pklname}_debug.pkl','wb') as f:
        pickle.dump(debugd,f)
    print("pkl saved")
    a,b,c = pickle.load(file=open(pklname, 'rb'))
    print("done")

#test
with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_nfl_minill3_test.json') as f:
#with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_politics_minill3_test.json') as f:
    d = json.loads(f.read())
    
#with open("/share_7/projects/hais/data/reddit_nfl/author_list_min20_13765_201809101112_20190102_nfl_minill3_test_top500.txt") as f:
#with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_201809101112_20190102_nfl_minill3_test.txt') as f:
with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_201809101112_20190102_nfl_minill3_test_top3000.txt') as f:
#with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_201809101112_20190102_politics_minill3_test.txt') as f:
#with open("/share_7/projects/hais/data/reddit_nfl/author_list_min20_19044_201809101112_20190102_politics_minill3_test_top500.txt") as f:
     u = f.read().strip().split('\n')

        
# create_data(d,u,
#             "tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl",
#             "/share_7/projects/hais/data/reddit_nfl_bertfp/minill3politics_data/201809101112_20190102_politics_minill3_test.tsv",
#             "/share_7/projects/hais/data/reddit_nfl_bertfp/minill3politics_data/201809101112_20190102_politics_minill3_test_authors.tsv",
#             1
#            )

create_data(d,u,
            "tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl",
            "/share_7/projects/hais/data/reddit_nfl_bertfp/minill3nfl10_data/201809101112_20190102_nfl_minill3_test.tsv",
            "/share_7/projects/hais/data/reddit_nfl_bertfp/minill3nfl10_data/201809101112_20190102_nfl_minill3_test_authors.tsv",
            1
           )


#train
with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_nfl_minill3_train.json') as f:
#with open('/share_7/projects/hais/data/reddit_nfl/201809101112_20190102_politics_minill3_train.json') as f:
    d = json.loads(f.read())

#with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_11503_201809101112_20190102_nfl_minill3_train_top500.txt') as f:
#with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_1983_201809101112_20190102_nfl_minill3_train.txt') as f:
#with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_201809101112_20190102_nfl_minill3_train.txt') as f:
with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_1983_201809101112_20190102_nfl_minill3_train.txt') as f:
#with open('/share_7/projects/hais/data/reddit_nfl/author_list_min20_14501_201809101112_20190102_politics_minill3_train.txt') as f:
#with open("/share_7/projects/hais/data/reddit_nfl/author_list_min20_14501_201809101112_20190102_politics_minill3_train_top500.txt") as f:
    u = f.read().strip().split('\n')
    

create_data(d, u, 
            'tm_train_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl',
            None,
            None,
            10)







In [ ]:
# output.show()

In [ ]:
#%%capture output
#nfl_minill3_min20_top3000
#politics_minill3_min30
#politics_minill3_min20
"""
!cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/dev_reddit.pkl
!cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/test_reddit.pkl
!cp tm_train_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/train_reddit.pkl
!cp vocab_and_embeddings_redpol.pkl dataset/reddit/vocab_and_embeddings.pkl
"""


# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/dev_reddit_nfl_minill3_min20_all.pkl
# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/test_reddit_nfl_minill3_min20_all.pkl
# !cp tm_train_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/train_reddit_nfl_minill3_min20_all.pkl
# !cp vocab_and_embeddings_nfl_minill3.pkl dataset/reddit/vocab_and_embeddings_nfl_minill3_min20_all.pkl



# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/dev_reddit_politics_minill3_min30.pkl
# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/test_reddit_politics_minill3_min30.pkl
# !cp tm_train_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/train_reddit_politics_minill3_min30.pkl
# !cp vocab_and_embeddings_politics_minill3.pkl dataset/reddit/vocab_and_embeddings_politics_minill3_min30.pkl



# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/dev_reddit_politics_minill3_min20.pkl
# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/test_reddit_politics_minill3_min20.pkl
# !cp tm_train_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/train_reddit_politics_minill3_min20.pkl
# !cp vocab_and_embeddings_politics_minill3.pkl dataset/reddit/vocab_and_embeddings_politics_minill3_min20.pkl

#slide10 history19 nfl min 20
# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/dev_reddit_nfl_minill3_min20_all_lc10.pkl
# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/test_reddit_nfl_minill3_min20_all_lc10.pkl
# !cp tm_train_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/train_reddit_nfl_minill3_min20_all_lc10.pkl
# !cp vocab_and_embeddings_nfl_minill3.pkl dataset/reddit/vocab_and_embeddings_nfl_minill3_min20_all_lc10.pkl

#slide10 history19 nfl min 20 top3000
# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/dev_reddit_nfl_minill3_min20_top3000_lc10.pkl
# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/test_reddit_nfl_minill3_min20_top3000_lc10.pkl
# !cp tm_train_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/train_reddit_nfl_minill3_min20_top3000_lc10.pkl
# !cp vocab_and_embeddings_nfl_minill3.pkl dataset/reddit/vocab_and_embeddings_nfl_minill3_min20_top3000_lc10.pkl

#slide10 history19 politics min20
# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/dev_reddit_politics_minill3_min20_all_lc10.pkl
# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/test_reddit_politics_minill3_min20_all_lc10.pkl
# !cp tm_train_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/train_reddit_politics_minill3_min20_all_lc10.pkl
# !cp vocab_and_embeddings_politics_minill3.pkl dataset/reddit/vocab_and_embeddings_politics_minill3_min20_all_lc10.pkl

# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/dev_reddit_politics_minill3_min20_top500_lc10.pkl
# !cp tm_test_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/test_reddit_politics_minill3_min20_top500_lc10.pkl
# !cp tm_train_redpol_min20_top10000_clean_bugfix_uniqhist_cemb.pkl dataset/reddit/train_reddit_politics_minill3_min20_top500_lc10.pkl
# !cp vocab_and_embeddings_politics_minill3.pkl dataset/reddit/vocab_and_embeddings_politics_minill3_min20_top500_lc10.pkl

!pwd
print("done.")


In [ ]:
# output.show()

In [ ]:
#%%capture output
# !cp tm_test_min20_clean_bugfix_uniqhist.pkl dataset/reddit2/dev_reddit2.pkl
# !cp tm_test_min20_clean_bugfix_uniqhist.pkl dataset/reddit2/test_reddit2.pkl
# !cp tm_train_min20_old_clean_uniqhist.pkl dataset/reddit2/train_reddit2.pkl
# print("done.")

In [ ]:
# output.show()

In [ ]:
# !pwd

# データ構造解説

- [ud,rd,yd]がpickleになってる
- ydは0 or 1のarray
- rdは当てに行く返答。n個に1個正解（yd=1）がある
- udはrdを当てるためのデータ。長さ29のarrayで中身は発話で、最後の要素以外はだれか（だれかは毎回異なって良い）の発話とそれに対するrdの返答者による返答となる。最後の要素はだれかの発話となり、rdの返答はこの最後のだれかの発話への返答となる。長さが余る場合は後ろから埋めて、あまりを空発話にする
- 返答・発話は、長さが固定（seqlen）のarrayで中身はtoken id(integer)。余る場合は先頭から<PAD>に対応するidで埋める。空発話であれば、全て<PAD>に対応するidとなる